In [25]:
import requests
import numpy as np
import pandas as pd
from transformers import pipeline


# Load Data

In [26]:
# Load the CSV file into a DataFrame
df_en = pd.read_csv('../Preprocessing/data_clean/labeled-data/labeled-en_clean_1-1_not101010.csv')

In [27]:
df_en.head()

,data,player,language,publishedAt,Label
0,ten if you included the toe-poked volley to te...,palacios,en,2023-02-16T23:56:00Z,NaN
1,bayerleverkusen took the lead again in the st ...,palacios,en,2023-02-23T20:50:50Z,NaN
2,wissam ben yedder levelled straight away from ...,palacios,en,2023-02-23T20:53:59Z,positiv
3,"midfielders: leandro paredes (juventus), angel...",palacios,en,2023-03-03T16:40:46Z,neutral
4,midfielders: rodrigo de paul (atletico madrid)...,palacios,en,2023-03-03T18:17:37Z,neutral


# Fix NA

In [28]:
#for DE datasate drop empty lines
df_de = df_en.dropna(subset=['data'])

In [29]:
df_en.head()

,data,player,language,publishedAt,Label
0,ten if you included the toe-poked volley to te...,palacios,en,2023-02-16T23:56:00Z,NaN
1,bayerleverkusen took the lead again in the st ...,palacios,en,2023-02-23T20:50:50Z,NaN
2,wissam ben yedder levelled straight away from ...,palacios,en,2023-02-23T20:53:59Z,positiv
3,"midfielders: leandro paredes (juventus), angel...",palacios,en,2023-03-03T16:40:46Z,neutral
4,midfielders: rodrigo de paul (atletico madrid)...,palacios,en,2023-03-03T18:17:37Z,neutral


# Sentiment analysis full text

In [30]:
sentiment_classifier = pipeline('sentiment-analysis', model='bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [48]:
def perform_comparison_sentiment_analysis(df):
    # Create empty lists to store the sentiment scores and labels
    sentiment_scores = []
    sentiment_labels = []

    # Iterate over the 'data' column in the DataFrame
    for text in df['data']:
        # Perform sentiment analysis using the Hugging Face pipeline
        result = sentiment_classifier(text)[0]
        sentiment_score = result['score']
        
        # Convert logits to predicted label (positive/negative)
        predicted_label = 1 if sentiment_score > 0.5 else 0
        sentiment_label = "positiv" if predicted_label == 1 else "negativ"

        # Append the sentiment score and label to the respective lists
        sentiment_scores.append(sentiment_score)
        sentiment_labels.append(sentiment_label)

    # Add the sentiment scores and labels as new columns in the DataFrame
    df['sentiment_score'] = sentiment_scores
    df['sentiment_label'] = sentiment_labels

    return df


# Perform sentiment analysis on the DataFrame
df = perform_comparison_sentiment_analysis(df_en)

# Print the updated DataFrame
df.head()


,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
2,wissam ben yedder levelled straight away from ...,palacios,en,2023-02-23T20:53:59Z,positiv,0.536872,positiv
3,"midfielders: leandro paredes (juventus), angel...",palacios,en,2023-03-03T16:40:46Z,neutral,0.558450,positiv
4,midfielders: rodrigo de paul (atletico madrid)...,palacios,en,2023-03-03T18:17:37Z,neutral,0.543187,positiv
5,a first half goal from joshua kimmich was canc...,palacios,en,2023-03-19T18:30:00Z,positiv,0.550972,positiv
9,""" palacios scored two penalties as bayerlever...",palacios,en,2023-03-19T20:03:28Z,positiv,0.532465,positiv


In [49]:
'''three labels:
def perform_comparison_sentiment_analysis(df):
    # Create empty lists to store the sentiment scores and labels
    sentiment_scores = []
    sentiment_labels = []

    # Iterate over the 'data' column in the DataFrame
    for text in df['data']:
        # Perform sentiment analysis using the Hugging Face pipeline
        result = sentiment_classifier(text)[0]
        sentiment_score = result['score']
        
        # Determine sentiment label based on sentiment score
        if sentiment_score > 0.6:
            sentiment_label = "positive"
        elif sentiment_score < 0.4:
            sentiment_label = "negative"
        else:
            sentiment_label = "neutral"

        # Append the sentiment score and label to the respective lists
        sentiment_scores.append(sentiment_score)
        sentiment_labels.append(sentiment_label)

    # Add the sentiment scores and labels as new columns in the DataFrame
    df['sentiment_score'] = sentiment_scores
    df['sentiment_label'] = sentiment_labels

    return df


# Perform sentiment analysis on the DataFrame
df = perform_comparison_sentiment_analysis(df_en)

# Print the updated DataFrame
df.head()

'''

'three labels:\ndef perform_comparison_sentiment_analysis(df):\n    # Create empty lists to store the sentiment scores and labels\n    sentiment_scores = []\n    sentiment_labels = []\n\n    # Iterate over the \'data\' column in the DataFrame\n    for text in df[\'data\']:\n        # Perform sentiment analysis using the Hugging Face pipeline\n        result = sentiment_classifier(text)[0]\n        sentiment_score = result[\'score\']\n        \n        # Determine sentiment label based on sentiment score\n        if sentiment_score > 0.6:\n            sentiment_label = "positive"\n        elif sentiment_score < 0.4:\n            sentiment_label = "negative"\n        else:\n            sentiment_label = "neutral"\n\n        # Append the sentiment score and label to the respective lists\n        sentiment_scores.append(sentiment_score)\n        sentiment_labels.append(sentiment_label)\n\n    # Add the sentiment scores and labels as new columns in the DataFrame\n    df[\'sentiment_score\']

In [50]:
print(df['sentiment_label'].unique())
print(df['sentiment_score'].min())
print(df['sentiment_score'].max())

['positiv']
0.5067351460456848
0.6003268957138062


In [51]:
df[df['sentiment_label'] == 'negativ'].shape[0]/df.shape[0]

0.0

In [52]:
# Drop rows where 'Label' is NaN or empty
df_en.dropna(subset=['Label'], inplace=True)

# Print the updated DataFrame
data = df_en[df_en['Label'] != 'neutral']

df_en.head()

,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
2,wissam ben yedder levelled straight away from ...,palacios,en,2023-02-23T20:53:59Z,positiv,0.536872,positiv
3,"midfielders: leandro paredes (juventus), angel...",palacios,en,2023-03-03T16:40:46Z,neutral,0.558450,positiv
4,midfielders: rodrigo de paul (atletico madrid)...,palacios,en,2023-03-03T18:17:37Z,neutral,0.543187,positiv
5,a first half goal from joshua kimmich was canc...,palacios,en,2023-03-19T18:30:00Z,positiv,0.550972,positiv
9,""" palacios scored two penalties as bayerlever...",palacios,en,2023-03-19T20:03:28Z,positiv,0.532465,positiv


In [53]:
data

,data,player,language,publishedAt,Label,sentiment_score,sentiment_label
2,wissam ben yedder levelled straight away from ...,palacios,en,2023-02-23T20:53:59Z,positiv,0.536872,positiv
5,a first half goal from joshua kimmich was canc...,palacios,en,2023-03-19T18:30:00Z,positiv,0.550972,positiv
9,""" palacios scored two penalties as bayerlever...",palacios,en,2023-03-19T20:03:28Z,positiv,0.532465,positiv
17,"today it was a lifesaver for the game."" palaci...",palacios,en,2023-03-20T21:20:28Z,positiv,0.522641,positiv
19,foul by palacios (argentina),palacios,en,2023-03-29T06:58:57Z,negativ,0.581393,positiv
24,the main area that bayerleverkusen have lent o...,frimpong,en,2023-03-20T13:00:35Z,positiv,0.536309,positiv
26,ronald koeman launched a scathing assessment o...,frimpong,en,2023-03-22T10:35:51Z,negativ,0.536700,positiv
27,"ronald koeman has made an undiplomatic return,...",frimpong,en,2023-03-24T07:23:27Z,negativ,0.547363,positiv
30,bayerleverkusen handed schalke their first def...,frimpong,en,2023-04-01T19:24:43Z,positiv,0.551380,positiv
55,the red devils have been linked with bayerleve...,frimpong,en,2023-05-24T11:33:33Z,positiv,0.529674,positiv


In [54]:
# Calculate the accuracy
accuracy = (data['sentiment_label'] == data['Label']).mean() * 100

# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy))

Accuracy: 76.47%
